In [45]:
import capytaine as cpt
import numpy as np
import matplotlib.pyplot as plt
from capytaine.bem.airy_waves import airy_waves_potential, airy_waves_velocity, froude_krylov_force
from capytaine.bem.solver import BEMSolver
from capytaine.green_functions.delhommeau import Delhommeau
from capytaine.bem.engines import BasicMatrixEngine
from capytaine.bem.problems_and_results import RadiationProblem
from capytaine.bodies.predefined.spheres import Sphere
cpt.__version__ #get_potential_on_mesh if not version 2.0



omega = 1.0
rho = 1000# density of our special material
wave_amp = 1
wave_num =  1.0/9.81

def generate_body(xyz):
    mesh1 = cpt.meshes.predefined.mesh_vertical_cylinder(radius=1, length=2, center=(xyz[0], xyz[1], 0))
    body = cpt.FloatingBody(mesh1)
    body.add_translation_dof(name='Heave')
    body = body.immersed_part()
    body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
    return body




#calculate angle theta_ij from centre of one body to other
def theta_ij(X,Y): 
    x1,y1= X[0],X[1]
    x2,y2 = Y[0], Y[1]
    
    if x1 ==x2 and y1==y2:
        return 0
    if x2==x1:
        theta = np.pi/2
    else:
        theta = np.arctan2((y2-y1),(x2-x1))
    return theta


def solve_forces(diff_result):
    
    """
    """
    return diff_result.forces



def get_neighbors(xyzees):
    neighbor = {xyz:[] for xyz in xyzees}
    for xyz in xyzees:
        for zyx in xyzees:
            if not xyz == zyx:
                neighbor[xyz].append(zyx)
    return neighbor






In [46]:
from capytaine.tools.lists_of_points import _normalize_points, _normalize_free_surface_points
import numpy as  np
def plane_waves_velocity(points2,pb,theta_ij):
    """Compute the fluid velocity for plane waves at a given point (or array of points).

    Parameters
    ----------
    points: array of shape (3) or (N x 3)
    points2: like points
        coordinates of the points in which to evaluate the potential.
    pb: DiffractionProblem
        problem with the environmental conditions (g, rho, ...) of interest

    Returns
    -------
    array of shape (3) or (N x 3)
        the velocity vectors
    """
    points = pb.body.mesh.faces_centers
 

    points, output_shape = _normalize_points(points)
    #print(f"shape of points in plane awave function{points.shape} and output shape {output_shape}")
    #points2 = _normalize_points(points2)
    x, y, z = points.T
  
    xj= np.array(points2[0])
    yj = np.array(points2[1])

    k = pb.wavenumber
    h = pb.water_depth
    pb.wave_direction = theta_ij

    wbar = (x-xj) * np.cos(pb.wave_direction) + (y-yj) * np.sin(pb.wave_direction)

 
    cih = np.exp(k*z)
    sih = np.exp(k*z)

    v = pb.g*k/pb.omega * \
        np.exp(1j * k * wbar) * \
        np.array([np.cos(pb.wave_direction) * cih, np.sin(pb.wave_direction) * cih, -1j * sih])
    

    bc_not_yet =  v.T.reshape((*output_shape, 3))
#BE CAREFUL WITH DIFFEERENT BODY..CURRRENTLY ITS SAME - should be neigboring body...we find the bc due to a source body to 
# target body...thus have to use target/negibor body
    pb.boundary_condition  = -(
                  bc_not_yet
                    * pb.body.mesh.faces_normals
            ).sum(axis=1)
   
    return  pb.boundary_condition 

from collections import defaultdict
import numpy as np

def gen_subseq_diff_prob(dictBC,body,loc_bodies):
  

    total_bc = defaultdict(list)

    for k, v in dictBC.items():
        for s, m in v.items():
            #print(f" shape of m {m.shape}")
            total_bc[k].append(m)

        
    total_bc = {k:np.sum(np.asarray(v),axis = 0) for k, v in total_bc.items()}
    
   # print("shape of bc in total_bc dict")
   
    prob = cpt.DiffractionProblem(body=body, water_depth = np.infty,
                                      omega=omega, wave_direction=0.)
    def setBC(diff,bc):
        diff.boundary_condition = bc 
        return diff
    new_diff_problem = setBC(prob,total_bc[loc_bodies[body]])
    #print("Afteer summation ")
  
    return new_diff_problem



In [47]:
xyzees = {(0,0,0),(10,10,0)}
#if using more bodies from above
#xyzees = {(x,y,0) for x,y in zip(x_flattened,y_flattened)}
#print(xyzees)
    
bodies = [generate_body(xyz) for xyz in xyzees ]
neighbors = get_neighbors(xyzees)
cache_engine = BasicMatrixEngine(matrix_cache_size=0)

solver = cpt.BEMSolver(engine=cache_engine)
loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}


diff_problems = {body:cpt.DiffractionProblem(body=body, water_depth = np.infty,
                                      omega=omega, wave_direction=0.) for body in bodies}
# print("Thee boundary condition shape usually is")
# for body,prob in diff_problems.items():
#     print(prob.boundary_condition.shape)

diff_loc= {generate_body(loc):loc for loc in xyzees }

loc_diff = {loc_bodies.get(body):diff for body,diff in diff_problems.items() }

rad_problems = {body: cpt.RadiationProblem(body=body, water_depth = np.infty,
                                      omega=omega) for body in bodies}

diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}


body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                     solver.compute_potential(np.array(body_neighbors_locs[body]),
                                                              diff_results[body]))))
                                       for body in bodies}
all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees }
thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}

# boundary condition due to the body at its neighbors
body_bc_at_neighbors = {body:{nbros:plane_waves_velocity(nbros,diff_problems[body],thetas[loc_bodies[body]][nbros]) for nbros in neighbors} for body in bodies}
# collecting all bc of a body at its location
all_other_bc_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_bc_at_neighbors.items()} for xyz in xyzees }
# for k,v in all_other_bc_each_loc.items():
#     for i,m in v.items():
#         print(m.shape)

In [48]:
N_bodies = 2 #len(xyzees)
max_iteration = 2 #(dead or alive lol)

# body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
#                                        airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}

iterate = 1
while iterate<=max_iteration:
    print(iterate)
    #go back to step 1 and solve the diff and radiation problems again
    #if it is slow it is running green's function again so we may need to cache it. now let's just go through with it
   # initially solve the regular diffraction problem with the usual air wave boundary condition
    diff_problems = {body:gen_subseq_diff_prob(all_other_bc_each_loc,body,loc_bodies) for body in bodies}

   #rad_problems = {body: cpt.RadiationProblem(body=body, water_depth = np.infty,
                      #                omega=omega) for body in bodies}
    
    diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
  #rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}
    body_bc_at_neighbors = {body:{nbros:plane_waves_velocity(nbros,diff_problems[body],thetas[loc_bodies[body]][nbros]) for nbros in neighbors} for body in bodies}
# collecting all bc of a body at its location
    all_other_bc_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_bc_at_neighbors.items()} for xyz in xyzees }
#
    iterate+=1
    

1
2


In [49]:
new_results = {loc_to_body.get(loc):solve_forces(diff_results[loc_to_body.get(loc)]) for loc,diff_prob in loc_diff.items()}
new_results

{FloatingBody(mesh=cylinder_276, dofs={Heave}, name=10_10_0): {'Heave': (-2295.6953609739267+1499.681784610071j)},
 FloatingBody(mesh=cylinder_282, dofs={Heave}, name=0_0_0): {'Heave': (-2295.695360973923+1499.6817846100735j)}}